In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive/


In [0]:
import torch
torch.cuda.is_available()

True

In [0]:
ls

gdrive/  sample_data/


In [0]:
import os
os.chdir("/content/gdrive/My Drive/BOUNDING_BOX")

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.utils.data
import os
from os.path import exists, join, split
from PIL import Image
import numpy as np

class LSUN(torch.utils.data.Dataset):

    def __init__(self, data_dir, phase, transforms, list_dir=None,
                 out_name=False):
        self.list_dir = data_dir if list_dir is None else list_dir
        self.data_dir = data_dir
        self.out_name = out_name
        self.phase = phase
        self.transforms = transforms
        self.image_list = None
        self.label_list = None
        self.bbox_list = None
        self.read_lists()

    def __getitem__(self, index):
        data = [Image.open(join(self.data_dir, self.image_list[index]))]
        data = np.array(data[0])
        if len(data.shape) == 2:
            data = np.stack([data , data , data] , axis = 2)
        data = [Image.fromarray(data)]
        # print(data)
        if self.label_list is not None:
       	    data.append(Image.open(join(self.data_dir, self.label_list[index])))
        data = list(self.transforms(*data))
        if self.out_name:
            if self.label_list is None:
                data.append(data[0][0, :, :])
            data.append(self.image_list[index])
        return tuple(data)

    def __len__(self):
        return len(self.image_list)

    def read_lists(self):
        image_path = join(self.list_dir, self.phase + '_images.txt')
        label_path = join(self.list_dir, self.phase + '_labels.txt')
        # print(image_path)
        assert exists(image_path)
        self.image_list = [line.strip() for line in open(image_path, 'r')]
        if exists(label_path):
            self.label_list = [line.strip() for line in open(label_path, 'r')]
            assert len(self.image_list) == len(self.label_list)

In [0]:
import numbers
import random

import numpy as np
from PIL import Image, ImageOps
import torch
'''
custom data transform for LSUN dataset
torchvision is not compatible with LSUN
'''

class RandomCrop(object):
    def __init__(self, size):
        if isinstance(size, numbers.Number):
            self.size = (int(size), int(size))
        else:
            self.size = size

    def __call__(self, image, label, *args):
        assert label is None or image.size == label.size, \
            "image and label doesn't have the same size {} / {}".format(
                image.size, label.size)

        w, h = image.size
        tw, th = self.size
        top = bottom = left = right = 0
        if w < tw:
            left = (tw - w) // 2
            right = tw - w - left
        if h < th:
            top = (th - h) // 2
            bottom = th - h - top
        if left > 0 or right > 0 or top > 0 or bottom > 0:
            label = pad_image(
                'constant', label, top, bottom, left, right, value=255)
            image = pad_image(
                'reflection', image, top, bottom, left, right)
        w, h = image.size
        if w == tw and h == th:
            return (image, label, *args)

        x1 = random.randint(0, w - tw)
        y1 = random.randint(0, h - th)
        results = [image.crop((x1, y1, x1 + tw, y1 + th))]
        if label is not None:
            results.append(label.crop((x1, y1, x1 + tw, y1 + th)))
        results.extend(args)
        return results


class RandomScale(object):
    def __init__(self, scale):
        if isinstance(scale, numbers.Number):
            scale = [1 / scale, scale]
        self.scale = scale

    def __call__(self, image, label):
        ratio = random.uniform(self.scale[0], self.scale[1])
        w, h = image.size
        tw = int(ratio * w)
        th = int(ratio * h)
        if ratio == 1:
            return image, label
        elif ratio < 1:
            interpolation = Image.ANTIALIAS
        else:
            interpolation = Image.CUBIC
        return image.resize((tw, th), interpolation), \
               label.resize((tw, th), Image.NEAREST)


class RandomRotate(object):
    """Crops the given PIL.Image at a random location to have a region of
    the given size. size can be a tuple (target_height, target_width)
    or an integer, in which case the target will be of a square shape (size, size)
    """

    def __init__(self, angle):
        self.angle = angle

    def __call__(self, image, label=None, *args):
        assert label is None or image.size == label.size

        w, h = image.size
        p = max((h, w))
        angle = random.randint(0, self.angle * 2) - self.angle

        if label is not None:
            label = pad_image('constant', label, h, h, w, w, value=255)
            label = label.rotate(angle, resample=Image.NEAREST)
            label = label.crop((w, h, w + w, h + h))

        image = pad_image('reflection', image, h, h, w, w)
        image = image.rotate(angle, resample=Image.BILINEAR)
        image = image.crop((w, h, w + w, h + h))
        return image, label


class RandomHorizontalFlip(object):
    """Randomly horizontally flips the given PIL.Image with a probability of 0.5
    """

    def __call__(self, image, label):
        if random.random() < 0.5:
            results = [image.transpose(Image.FLIP_LEFT_RIGHT),
                       label.transpose(Image.FLIP_LEFT_RIGHT)]
        else:
            results = [image, label]
        return results


class Normalize(object):
    """Given mean: (R, G, B) and std: (R, G, B),
    will normalize each channel of the torch.*Tensor, i.e.
    channel = (channel - mean) / std
    """

    def __init__(self, mean, std):
        self.mean = torch.FloatTensor(mean)
        self.std = torch.FloatTensor(std)

    def __call__(self, image, label=None):
        for t, m, s in zip(image, self.mean, self.std):
            t.sub_(m).div_(s)
        if label is None:
            return image,
        else:
            return image, label


def pad_reflection(image, top, bottom, left, right):
    if top == 0 and bottom == 0 and left == 0 and right == 0:
        return image
    h, w = image.shape[:2]
    next_top = next_bottom = next_left = next_right = 0
    if top > h - 1:
        next_top = top - h + 1
        top = h - 1
    if bottom > h - 1:
        next_bottom = bottom - h + 1
        bottom = h - 1
    if left > w - 1:
        next_left = left - w + 1
        left = w - 1
    if right > w - 1:
        next_right = right - w + 1
        right = w - 1
    new_shape = list(image.shape)
    new_shape[0] += top + bottom
    new_shape[1] += left + right
    new_image = np.empty(new_shape, dtype=image.dtype)
    new_image[top:top+h, left:left+w] = image
    new_image[:top, left:left+w] = image[top:0:-1, :]
    new_image[top+h:, left:left+w] = image[-1:-bottom-1:-1, :]
    new_image[:, :left] = new_image[:, left*2:left:-1]
    new_image[:, left+w:] = new_image[:, -right-1:-right*2-1:-1]
    return pad_reflection(new_image, next_top, next_bottom,
                          next_left, next_right)


def pad_constant(image, top, bottom, left, right, value):
    if top == 0 and bottom == 0 and left == 0 and right == 0:
        return image
    h, w = image.shape[:2]
    new_shape = list(image.shape)
    new_shape[0] += top + bottom
    new_shape[1] += left + right
    new_image = np.empty(new_shape, dtype=image.dtype)
    new_image.fill(value)
    new_image[top:top+h, left:left+w] = image
    return new_image


def pad_image(mode, image, top, bottom, left, right, value=0):
    if mode == 'reflection':
        return Image.fromarray(
            pad_reflection(np.asarray(image), top, bottom, left, right))
    elif mode == 'constant':
        return Image.fromarray(
            pad_constant(np.asarray(image), top, bottom, left, right, value))
    else:
        raise ValueError('Unknown mode {}'.format(mode))


class Pad(object):
    """Pads the given PIL.Image on all sides with the given "pad" value"""

    def __init__(self, padding, fill=0):
        assert isinstance(padding, numbers.Number)
        assert isinstance(fill, numbers.Number) or isinstance(fill, str) or \
               isinstance(fill, tuple)
        self.padding = padding
        self.fill = fill

    def __call__(self, image, label=None, *args):
        if label is not None:
            label = pad_image(
                'constant', label,
                self.padding, self.padding, self.padding, self.padding,
                value=255)
        if self.fill == -1:
            image = pad_image(
                'reflection', image,
                self.padding, self.padding, self.padding, self.padding)
        else:
            image = pad_image(
                'constant', image,
                self.padding, self.padding, self.padding, self.padding,
                value=self.fill)
        return (image, label, *args)


class PadImage(object):
    def __init__(self, padding, fill=0):
        assert isinstance(padding, numbers.Number)
        assert isinstance(fill, numbers.Number) or isinstance(fill, str) or \
               isinstance(fill, tuple)
        self.padding = padding
        self.fill = fill

    def __call__(self, image, label=None, *args):
        if self.fill == -1:
            image = pad_image(
                'reflection', image,
                self.padding, self.padding, self.padding, self.padding)
        else:
            image = ImageOps.expand(image, border=self.padding, fill=self.fill)
        return (image, label, *args)


class ToTensor(object):
    """Converts a PIL.Image or numpy.ndarray (H x W x C) in the range
    [0, 255] to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0].
    """

    def __call__(self, pic, label=None):
        if isinstance(pic, np.ndarray):
            # handle numpy array
            img = torch.from_numpy(pic)
        else:
            # handle PIL Image
            img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
            # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
            if pic.mode == 'YCbCr':
                nchannel = 3
            else:
                nchannel = len(pic.mode)
            img = img.view(pic.size[1], pic.size[0], nchannel)
            # put it from HWC to CHW format
            # yikes, this transpose takes 80% of the loading time/CPU
            img = img.transpose(0, 1).transpose(0, 2).contiguous()
        img = img.float().div(255)
        if label is None:
            return img,
        else:
            return img, torch.LongTensor(np.array(label, dtype=np.int))


class Compose(object):
    """Composes several transforms together.
    """

    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, *args):
        for t in self.transforms:
            args = t(*args)
        return args

In [0]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim
# from torchsummaryX import summary
from torch.nn import functional as F
from collections import OrderedDict

class PreTrainedResNet(nn.Module):

  def __init__(self, feature_extracting, num_classes=4):
    super(PreTrainedResNet, self).__init__()
    self.fcn = models.segmentation.fcn_resnet101(pretrained=True)
    if feature_extracting:
      for param in self.fcn.parameters():
          param.requires_grad = False     # Fine tune whole network if requires_grad = true
                                          # or use slower learning rate for earlier layers
    #modified from models.segmentation.fcn.FCNHead, see
    #https://github.com/pytorch/vision/blob/master/torchvision/models/segmentation/fcn.py
    self.fcn.classifier = nn.Sequential(
      nn.Conv2d(2048, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
      nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
      nn.ReLU(),
      nn.Dropout(p=0.1),
      nn.Conv2d(128, num_classes, kernel_size=(1, 1), stride=(1, 1)),
    )

    self.fcn.aux_classifier = nn.Sequential(
      nn.Conv2d(1024, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False),
      nn.BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
      nn.ReLU(),
      nn.Dropout(p=0.1),
      nn.Conv2d(128, num_classes, kernel_size=(1, 1), stride=(1, 1)),
    )
    # self.fcn.classifier = models.segmentation.fcn.FCNHead(2048,4)
    # self.fcn.aux_classifier = models.segmentation.fcn.FCNHead(1024,4)
    self.softmax = nn.LogSoftmax()
    # summary(self.fcn,torch.zeros((1,3,480,480)))

  def forward(self, x):
      input_shape = x.shape[-2:]
      # contract: features is a dict of tensors
      features = self.fcn.backbone(x)

      result = OrderedDict()
      x = features["out"]
      x = self.fcn.classifier(x)
#       x = F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
      result["out"] = x

      if self.fcn.aux_classifier is not None:
          x = features["aux"]
          x = self.fcn.aux_classifier(x)
#           x = F.interpolate(x, size=input_shape, mode='bilinear', align_corners=False)
          result["aux"] = x
      return result

  def optim_base_parameters(self, memo=None):
    for param in self.fcn.backbone.parameters():
        yield param

  def optim_seg_parameters(self, memo=None):
    for param in self.fcn.classifier.parameters():
        yield param
    for param in self.fcn.aux_classifier.parameters():
        yield param

In [0]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as trans
from torch.autograd import Variable
import os
import matplotlib.pyplot as plt
import logging
import cv2
import time
from matplotlib.colors import ListedColormap
NUM_EPOCHS = 100
LEARNING_RATE = 1e-2
LEARNING_RATE =  0.060611 / 10
BATCH_SIZE = 16
IM_SIZE = 240
CROP_SIZE = 244
FEATURE_EXTRACTING = False
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
R_ROT = 15
R_SCALE = 1.75
GPU = True
scriptdir = "/content/gdrive/My Drive/BOUNDING_BOX"
OUTPUT_DIR = 'output/'
DATADIR = 'data/lsun/'
def load_data():
    train_transform = Compose(
        [RandomRotate(R_ROT),
        RandomScale(R_SCALE),
        RandomCrop(IM_SIZE),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    val_transform = Compose(
        [RandomCrop(IM_SIZE),
        RandomHorizontalFlip(),
        ToTensor(),
        Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

    trainset = LSUN(DATADIR, 'train', train_transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                            shuffle=True, num_workers=8, pin_memory=True, drop_last=True)

    print("Train set size: "+str(len(trainset)))

    valset = LSUN(DATADIR, 'val', val_transform)
    valloader = torch.utils.data.DataLoader(valset, batch_size=BATCH_SIZE,
                                            shuffle=False, num_workers=8, pin_memory=True, drop_last=True)
    print("Val set size: "+str(len(valset)))
    return trainloader, valloader

def train(resume=False,pretrained=True):
    start = time.time()
    trainloader, valloader = load_data()
    # test(trainloader)
    model = PreTrainedResNet(FEATURE_EXTRACTING, num_classes=4)
    filename = "ckpt" + '.pth.tar'
    save_path = os.path.join(scriptdir,OUTPUT_DIR,filename)
    if resume: 
      checkpoint = torch.load(save_path) 
      model.load_state_dict(checkpoint['state'])
    # show_network(model)
    if GPU: model = model.cuda()
    criterion = nn.CrossEntropyLoss(ignore_index=255)
    if GPU: criterion.cuda()
    if FEATURE_EXTRACTING:
        optimizer = torch.optim.SGD(model.parameters(),lr=LEARNING_RATE*10, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
    else:
        optimizer = torch.optim.SGD([{'params':model.optim_base_parameters(), 'lr':LEARNING_RATE},
                                {'params':model.optim_seg_parameters(),'lr':LEARNING_RATE*10}],
                                momentum=MOMENTUM,
                                weight_decay=WEIGHT_DECAY)

    #learning rate decay
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, int(NUM_EPOCHS * 1), eta_min = 1e-6)

    #to resume learning:
    start_ep = 0
    if resume: start_ep = checkpoint['epoch']
    for epoch in range(start_ep, NUM_EPOCHS):
        if not FEATURE_EXTRACTING: base_lr, lr = scheduler.get_lr()
        if FEATURE_EXTRACTING: lr = scheduler.get_lr()[0]
#         print(lr)
        logger.info('Epoch: [{0}]\tlr {1:.06f}'.format(epoch, lr))
        train_helper(trainloader, optimizer, model, criterion)
        scheduler.step()
        validate(valloader, optimizer, model, criterion, epoch)
        #save model
        state = {'epoch': epoch + 1,
                 'state': model.state_dict()}
        torch.save(state, save_path)
    end = time.time()
    print("time: ", end - start)

def train_helper(trainloader, optimizer, model, criterion):
    model.train()
    for i, (img, target) in enumerate(trainloader):
        # transform target into more pixelized layout image
        # not needed atm, res101 interpolates output


        small_target = torch.zeros(int(target.size(0)) , int(target.size(1)/8) , int(target.size(2)/8))
        for index in range(target.size(0)):
            temp = target[index,:,:]
            temp = cv2.resize(temp.numpy(),(int(target.size(1)/8) , int(target.size(2)/8)), interpolation=cv2.INTER_NEAREST)
            temp = torch.Tensor(temp)
            small_target[index,:,:] = temp
        target = small_target
        target = target.long()
        if GPU:
            target = target.cuda(async=True)
            img = img.cuda()
        input_var = Variable(img)
        target_var = Variable(target)
        # print("here")
        # print(img.size())
        outputs = model(input_var)
        output = outputs['out']
        aux_output = outputs['aux']
        # print(output.size())
        # print(aux_output.size())       
        loss1 = criterion(output,target_var)
        loss2 = criterion(aux_output,target_var)
        loss = loss1 + loss2
        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("Loss: ", loss.item())

def validate(valloader, optimizer, model, criterion, epoch):
    epoch_loss = 0.0
    model.eval()
    for i, (img, target) in enumerate(valloader):
        # transform target into more pixelized layout image
        # not needed atm, res101 interpolates output

        small_target = torch.zeros(int(target.size(0)) , int(target.size(1)/8) , int(target.size(2)/8))
        for index in range(target.size(0)):
            temp = target[index,:,:]
            temp = cv2.resize(temp.numpy(),(int(target.size(1)/8) , int(target.size(2)/8)), interpolation=cv2.INTER_NEAREST)
            temp = torch.Tensor(temp)
            small_target[index,:,:] = temp
        target = small_target
        target = target.long()
        with torch.no_grad():
            if GPU:
                target = target.cuda(async=True)
                img = img.cuda()
            input_var = Variable(img)
            target_var = Variable(target)
            
            outputs = model(input_var)
            output = outputs['out']
            aux_output = outputs['aux']
            loss1 = criterion(output,target_var)
            loss2 = criterion(aux_output,target_var)
            loss = loss1 + loss2
            epoch_loss += loss.item()
            if i % 300 == 0:
#               print("here")
              save_output_images(output, OUTPUT_DIR, epoch)
    print(epoch_loss)

def show_network(model):
    child_counter = 0
    for child in model.children():
        print(" child", child_counter, "is:")
        print(child)
        child_counter += 1
    # print(model.fcn.backbone)

def test(trainloader):
    dataiter = iter(trainloader)
    images, labels = dataiter.next()
    print(labels.size())
    showtensor(labels)

def showtensor(tensor):
    x = torch.narrow(tensor,0,0,1)
    plt.figure()
    plt.imshow(x.squeeze().numpy())
    plt.show()

def save_checkpoint(self, state):
        print("[*] Saving model to {}".format(self.ckpt_dir))

        filename = self.get_model_name() + '_ckpt.pth.tar'
        ckpt_path = os.path.join(self.ckpt_dir, filename)
        torch.save(state, ckpt_path)


In [0]:
def save_output_images(out, output_dir, epoch):
    """
    Saves a given (B x C x H x W) into an image file.
    If given a mini-batch tensor, will save the tensor as a grid of images.
    """
    # pdb.set_trace()
    _, pred = torch.max(out, 1)
    cmap = ListedColormap(['b', 'c', 'y', 'r'])
    pred = pred.cpu().data.numpy()
    for i in range(4):
#       im = Image.fromarray(pred[i].astype(np.uint8))
      fn = os.path.join(scriptdir, output_dir, str(epoch) + str(i) + '.png')
      plt.imsave(fn, pred[i], cmap=cmap)
#       out_dir = split(fn)[0]
#       print(out_dir)
#       print("here2")
#       im.save(fn)

In [0]:
logging.basicConfig(format = '[%(asctime)-15s %(filename)s:%(lineno)d %(funcName)s] %(message)s')
logger = logging.getLogger(__name__)
logger.setLevel(level=logging.DEBUG)
train(resume=True)

Train set size: 4000
Val set size: 394


Downloading: "https://download.pytorch.org/models/resnet101-5d3b4d8f.pth" to /root/.cache/torch/checkpoints/resnet101-5d3b4d8f.pth
100%|██████████| 178728960/178728960 [00:03<00:00, 47152363.02it/s]
Downloading: "https://download.pytorch.org/models/fcn_resnet101_coco-7ecb50ca.pth" to /root/.cache/torch/checkpoints/fcn_resnet101_coco-7ecb50ca.pth
100%|██████████| 217800805/217800805 [00:04<00:00, 53300713.12it/s]
[2019-07-26 21:25:04,108 <ipython-input-8-7896f9ef806a>:86 train] Epoch: [53]	lr 0.060611


Loss:  0.5587717294692993
Loss:  1.0591323375701904
Loss:  0.8005150556564331
23.669366896152496


[2019-07-26 21:38:58,387 <ipython-input-8-7896f9ef806a>:86 train] Epoch: [54]	lr 0.060581


Loss:  0.8489469289779663
Loss:  0.6324933767318726
Loss:  0.6755504608154297
23.59914129972458


...

Loss:  0.6690472364425659
Loss:  0.638717532157898
Loss:  0.4562562108039856
25.46927058696747


[2019-07-27 07:14:19,579 <ipython-input-8-7896f9ef806a>:86 train] Epoch: [98]	lr 0.034133


Loss:  0.4710877537727356
Loss:  0.632636547088623
Loss:  0.4761693775653839
27.240716874599457


[2019-07-27 07:27:23,526 <ipython-input-8-7896f9ef806a>:86 train] Epoch: [99]	lr 0.033187


Loss:  0.7405450940132141
Loss:  0.762008547782898
Loss:  0.8743240833282471
26.99716466665268
time:  36945.46297764778
